In [3]:
#Importamos las  librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn import  metrics
from sklearn.metrics import accuracy_score

### Paso 1: Leemos el Dataset

In [4]:
dataset_estadisticas_definitivo = pd.read_csv(r"https://raw.githubusercontent.com/PabloLedo97/Datasets-Master-Big-Data/main/DatasetDefinitivo_Pablo_Ledo.csv",sep=";",encoding='latin-1');
dataset_estadisticas_definitivo

,Equipo,Local/Visitante,Goles_Locales,Suma_Goles_Locales_Acumulados,Goles_Visitantes,Suma_Goles_Visitantes_Acumulados,Resultados,Fecha,Temporada
0,Málaga,Local,0,0,0,0,x,21/08/2015,2015/2016
1,Sevilla,Visitante,0,0,0,0,x,21/08/2015,2015/2016
2,Atlético,Local,1,1,0,0,1,22/08/2015,2015/2016
3,Las Palmas,Visitante,0,0,0,0,1,22/08/2015,2015/2016
4,Espanyol,Local,1,1,0,0,1,22/08/2015,2015/2016
...,...,...,...,...,...,...,...,...,...
5315,Villarreal,Visitante,0,0,2,20,2,22/05/2022,2021/2022
5316,R. Sociedad,Local,1,16,0,0,2,22/05/2022,2021/2022
5317,Atlético,Visitante,0,0,2,32,2,22/05/2022,2021/2022
5318,Sevilla,Local,1,36,0,0,1,22/05/2022,2021/2022


### Paso 2: Definimos  la matriz de entrenamiento y el conjunto de prueba

In [5]:
#Definimos las columnas del dataset que van a participar en la matriz
goles_locales = dataset_estadisticas_definitivo.Goles_Locales.tolist()
goles_locales_acumulados = dataset_estadisticas_definitivo.Suma_Goles_Locales_Acumulados.tolist()
goles_visitantes = dataset_estadisticas_definitivo.Goles_Visitantes.tolist()
goles_visitantes_acumulados = dataset_estadisticas_definitivo.Suma_Goles_Visitantes_Acumulados.tolist()

#Generamos la matriz
matriz =np.array([goles_locales,goles_locales_acumulados,goles_visitantes,goles_visitantes_acumulados])


#Generamos el vector con el conjunto de datos que queremos predecir
vector = dataset_estadisticas_definitivo.Resultados.tolist()

### Paso 3: División de los datos en conjunto de entrenamiento y test

In [6]:
#Conjunto de entrenamiento
X_train = matriz.transpose()   
y_train = vector

X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size= 0.2,random_state = 123)

### Paso 4:  Definición Modelo

In [7]:
modelo = XGBClassifier()

### Paso 5:  Entrenamiento del Modelo

In [8]:
modelo.fit(X_train, y_train)

C:\Users\usuario\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:05:52] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

### Paso 6: Realizamos la predicción

In [9]:
predicciones = modelo.predict(X = X_test)
predicciones

array(['x', '1', '2', ..., '1', '1', '1'], dtype='<U1')

### Paso 7: Evaluación modelo

#### Cálculo accuracy

In [10]:
accuracy = accuracy_score(y_true= y_test,y_pred= predicciones,normalize = True)
print(f"El accuracy de test es: {100 * accuracy} %")

El accuracy de test es: 60.6203007518797 %


#### Cálculo Matriz de Confusión

In [11]:
from sklearn.metrics import confusion_matrix

matriz_confusion = confusion_matrix(y_test,predicciones)
print(matriz_confusion)

[[386  35  81]
 [ 54 198  51]
 [111  87  61]]


#### Clasification Report

In [12]:
from sklearn.metrics import classification_report

reporte_clasificacion = classification_report(y_test, predicciones)

print(reporte_clasificacion)

              precision    recall  f1-score   support

           1       0.70      0.77      0.73       502
           2       0.62      0.65      0.64       303
           x       0.32      0.24      0.27       259

    accuracy                           0.61      1064
   macro avg       0.55      0.55      0.55      1064
weighted avg       0.58      0.61      0.59      1064

